In [1]:
from pickletools import optimize
import torch, torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import os
from skimage import io, transform

from model import *
from utils import predict_transform, bbox_iou
from dataset import filter_labels, DetectionDataset, Rescale, ToTensor, Normalise

plt.rcParams['figure.figsize'] = [15,15]

## Rejig Custom Dataset

**Objective:** We want each label to be [x, y, w, h, obj?, c_1, c_2, ..., c_n]

### First find out n_unique classes/categories

In [2]:
classes = load_classes("data/bdd100k.names")

## Load Dataset

### Testing manual pad transform

In [4]:
class Pad(object):
    """
    Add padding to image

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple)) # make sure output size is EITHER int or tuple
        self.output_size = output_size

    def __call__(self, sample):
        image, categories, bboxes = sample["image"], sample["categories"], sample["bboxes"]

        img_w, img_h = image.shape[1], image.shape[0]
        w, h = self.output_size

        # calculate new width and height
        new_w = int(img_w * min(w/img_w, h/img_h))
        new_h = int(img_h * min(w/img_w, h/img_h))
        resized_image = cv2.resize(image, (new_w,new_h), interpolation = cv2.INTER_CUBIC)
        
        canvas = np.full((self.output_size[1], self.output_size[0], 3), 128)

        canvas[(h-new_h)//2:(h-new_h)//2 + new_h,(w-new_w)//2:(w-new_w)//2 + new_w,  :] = resized_image

        return {"image": canvas, "categories": categories, "bboxes": bboxes}

In [5]:
filtered_labels = filter_labels("det_train_shortened.json")

train_data = DetectionDataset(
    label_dict=filtered_labels,
    root_dir='images/',
)
## load custom dataset + transforms
transformed_train_data = DetectionDataset(
    label_dict=filtered_labels,
    root_dir='images/',
    transform=transforms.Compose([
        Rescale(608),
        Normalise(0.5, 0.5),
        Pad((608,608)),
        ToTensor()
    ])
)

## dataloader
train_loader = DataLoader(
    transformed_train_data,
    batch_size=1,
    shuffle=True,
    num_workers=0
)

## Conv -> Detection (Predict Transform)

In [29]:
blocks = parse_cfg('./cfg/model.cfg')
net_info, module_list = create_modules(blocks)

In [30]:
modules = blocks[1:]

In [31]:
anchors = module_list[-1][0].anchors
anchors

[(10, 13), (16, 30), (33, 23)]

In [32]:
in_dims = int(net_info["height"])
in_dims

608

In [33]:
n_classes = int(modules[-1]["classes"])
n_classes

80

In [34]:
prediction = torch.load("yolo_layer_input.pt")
prediction.shape

torch.Size([1, 255, 52, 52])

In [35]:
batch_size = prediction.shape[0]
stride = in_dims // prediction.shape[2]
grid_size = prediction.shape[2]
bbox_attrs = 5 + n_classes
n_anchors = len(anchors)

In [36]:
prediction = prediction.view(batch_size, bbox_attrs*n_anchors, grid_size*grid_size)
prediction.shape

torch.Size([1, 255, 2704])

In [37]:
prediction = prediction.transpose(1,2).contiguous()
prediction.shape

torch.Size([1, 2704, 255])

In [38]:
prediction = prediction.view(batch_size, grid_size*grid_size*n_anchors, bbox_attrs)
prediction.shape

torch.Size([1, 8112, 85])

In [39]:
anchors = [(a[0]/stride, a[1]/stride) for a in anchors]
anchors

[(0.9090909090909091, 1.1818181818181819),
 (1.4545454545454546, 2.727272727272727),
 (3.0, 2.090909090909091)]

In [54]:
prediction[:,:,0] = torch.sigmoid(prediction[:,:,0]) # x - first index of each bbox attr row
prediction[:,:,1] = torch.sigmoid(prediction[:,:,1]) # y
prediction[:,:,4] = torch.sigmoid(prediction[:,:,4]) # obj confidence score

In [42]:
grid = np.arange(grid_size)
a,b = np.meshgrid(grid, grid)

In [61]:
x_offset = torch.FloatTensor(a).view(-1,1) # size (grid_h*grid_w, 1)
y_offset = torch.FloatTensor(b).view(-1,1) # size (grid_h*grid_w, 1)

In [70]:
x_offset

tensor([[ 0.],
        [ 1.],
        [ 2.],
        ...,
        [49.],
        [50.],
        [51.]])

In [81]:
x_y_offset = torch.cat((x_offset, y_offset), 1).repeat(1, n_anchors).view(-1, 2).unsqueeze(0)
x_y_offset

tensor([[[ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         ...,
         [51., 51.],
         [51., 51.],
         [51., 51.]]])

In [85]:
prediction[:,:,:2]

tensor([[[0.4176, 0.1791],
         [0.4340, 0.5981],
         [0.4927, 0.3315],
         ...,
         [0.5634, 0.3973],
         [0.4281, 0.2278],
         [0.3619, 0.5615]]])

In [87]:
prediction[:,:,:2] += x_y_offset

In [88]:
prediction[:,:,:2]

tensor([[[ 0.4176,  0.1791],
         [ 0.4340,  0.5981],
         [ 0.4927,  0.3315],
         ...,
         [51.5634, 51.3973],
         [51.4281, 51.2278],
         [51.3619, 51.5615]]])

In [92]:
anchors = torch.FloatTensor(anchors)
anchors

tensor([[0.9091, 1.1818],
        [1.4545, 2.7273],
        [3.0000, 2.0909]])

In [95]:
anchors.shape

torch.Size([3, 2])

In [99]:
anchors = anchors.repeat(grid_size*grid_size, 1).unsqueeze(0)

In [101]:
prediction[:,:,2:4] = torch.exp(prediction[:,:,2:4])*anchors

In [ ]:
prediction[:,:,5:5+n_classes] = torch.sigmoid((prediction[:,:,5:5+n_classes]))

In [105]:
prediction[:,:,:4] = prediction[:,:,:4]*stride
prediction

tensor([[[4.5941e+00, 1.9698e+00, 8.1939e+00,  ..., 6.1947e-04,
          9.8315e-04, 6.9588e-04],
         [4.7735e+00, 6.5795e+00, 6.2579e+00,  ..., 2.3491e-03,
          3.1130e-03, 1.6503e-03],
         [5.4195e+00, 3.6469e+00, 9.8375e+01,  ..., 1.2637e-03,
          1.7293e-03, 1.9966e-03],
         ...,
         [5.6720e+02, 5.6537e+02, 3.3548e+00,  ..., 3.9487e-05,
          4.6984e-05, 2.7504e-05],
         [5.6571e+02, 5.6351e+02, 7.4772e+00,  ..., 2.8264e-04,
          3.1384e-04, 2.6532e-04],
         [5.6498e+02, 5.6718e+02, 5.6728e+01,  ..., 1.7386e-04,
          2.2903e-04, 3.1671e-04]]])

In [106]:
prediction.shape

torch.Size([1, 8112, 85])

In [108]:
torch.load("pred.pt")

tensor([[[1.5642e+01, 1.1840e+01, 9.4082e+01,  ..., 6.6353e-04,
          9.0433e-04, 5.0939e-04],
         [1.8059e+01, 1.4927e+01, 1.0423e+02,  ..., 1.8236e-04,
          1.0421e-03, 1.3603e-03],
         [2.1244e+01, 1.2575e+01, 3.8758e+02,  ..., 3.3734e-03,
          6.7615e-03, 5.6956e-03],
         ...,
         [4.1251e+02, 4.1118e+02, 3.3548e+00,  ..., 3.9487e-05,
          4.6984e-05, 2.7504e-05],
         [4.1142e+02, 4.0982e+02, 7.4772e+00,  ..., 2.8264e-04,
          3.1384e-04, 2.6532e-04],
         [4.1090e+02, 4.1249e+02, 5.6728e+01,  ..., 1.7386e-04,
          2.2903e-04, 3.1671e-04]]])

## Define Network

In [6]:
net = Net(cfgfile="cfg/model.cfg")

In [7]:
class TestNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)     # in 1 , out 6, 5x5 kernel
        self.conv2 = nn.Conv2d(6, 16, 5)    # in 6, out 16, 5x5 kernel

        self.pool = nn.MaxPool2d(2, 2)      # this time define own maxpooling
        
        # an affine op: y = Wx + b
        self.fc1 = nn.Linear(16*5*5, 120)   # 5x5 is image dim
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # max pooling over a (2,2) window
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dims except batch dim
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # no activation on final layer -> output
        return x
testnet = TestNet()

## Define Loss Function and Optimiser

In [7]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
criterion = ...

## Train Network

In [ ]:
CUDA = torch.cuda.is_available()
for i, data in enumerate(train_loader):
    input_img, cat, bboxes = data.values()

    optimizer.zero_grad()
    #plt.imshow(input_img)
    #input_img = norm_with_padding(input_img, 608)
    outputs = net(Variable(input_img.float()), CUDA)
    

In [10]:
outputs[...,0] # select first bbox attr from all bbox tensors

tensor([[ 16.8364,  15.2165,  15.7572,  ..., 605.3627, 603.0162, 604.4326]])

## Troubleshooting bboxes

In [96]:
outputs = torch.load("1.pt")

In [97]:
outputs = outputs*(outputs[:,:,4] > 0.5).float().unsqueeze(2)

In [98]:
outputs.shape

torch.Size([1, 10647, 85])

In [2]:
10647/3

3549.0

In [99]:
box_corner = outputs.new(outputs.shape)
box_corner[:,:,0] = (outputs[:,:,0] - outputs[:,:,2]/2)
box_corner[:,:,1] = (outputs[:,:,1] - outputs[:,:,3]/2)
box_corner[:,:,2] = (outputs[:,:,0] + outputs[:,:,2]/2)
box_corner[:,:,3] = (outputs[:,:,1] + outputs[:,:,3]/2)
# replace in the prediction tensor
outputs[:,:,:4] = box_corner[:,:,:4]

In [100]:
image_pred = outputs[0] # for each image in batch

In [101]:
image_pred.shape

torch.Size([10647, 85])

In [102]:
image_pred[:,5:5+80].shape

torch.Size([10647, 80])

In [103]:
torch.max(torch.tensor([1, 2, 3, 4, 5]), 0)

torch.return_types.max(
values=tensor(5),
indices=tensor(4))

In [104]:
max_conf, max_conf_idx = torch.max(image_pred[:,5:5+80], 1)

In [105]:
max_conf.shape

torch.Size([10647])

In [106]:
max_conf = max_conf.float().unsqueeze(1) # max_conf (1, 1, 85)
max_conf_score = max_conf_idx.float().unsqueeze(1) # max_conf_score (1, 1, 85)

In [107]:
image_pred[:,:5].shape

torch.Size([10647, 5])

In [108]:
max_conf.shape

torch.Size([10647, 1])

In [109]:
max_conf_score.shape

torch.Size([10647, 1])

In [110]:
seq = (image_pred[:,:5], max_conf, max_conf_score)
image_pred = torch.cat(seq, 1)
image_pred.shape

torch.Size([10647, 7])

In [111]:
image_pred[:,4]

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [112]:
non_zero_idx = (torch.nonzero(image_pred[:, 4]))

In [113]:
non_zero_idx.shape

torch.Size([1, 1])

In [114]:
image_pred_ = image_pred[non_zero_idx.squeeze(), :].view(-1,7)

In [115]:
image_pred_

tensor([[-17.8275, 275.9133, 422.6099, 319.4225,   0.9276,   0.9997,   2.0000]])

In [122]:
conf_sort_idx = torch.sort(image_pred_[:,4], descending=True)[1]
image_pred_ordered = image_pred_[conf_sort_idx]

In [123]:
nms_conf = 0.4

conf_sort_idx = torch.sort(image_pred_[:,4], descending=True)[1]
image_pred_ordered = image_pred_[conf_sort_idx]

n_dets = image_pred_ordered.shape[0]
for idx in range(n_dets):
    # print(idx)
    # print(pred)
    try:
        ious = bbox_iou(image_pred_ordered[idx].unsqueeze(0), image_pred_ordered[idx+1:])
    except ValueError:
        break

    except IndexError:
        break

    iou_mask = (ious < nms_conf).float().unsqueeze(1)
    image_pred_ordered[idx+1:] *= iou_mask

    non_zero_idx = torch.nonzero(image_pred_ordered[:,4]).squeeze()
    image_pred_ordered = image_pred_ordered[non_zero_idx].view(-1,7)
    # print(image_pred_ordered)
    # print(image_pred_ordered.shape)

In [124]:
image_pred_ordered

tensor([[-17.8275, 275.9133, 422.6099, 319.4225,   0.9276,   0.9997,   2.0000]])

In [125]:
batch_idx = image_pred_ordered.new(image_pred_ordered.size(0), 1).fill_(idx)
batch_idx

tensor([[0.]])

In [ ]:
seq = batch_idx, image_pred_ordered
output = torch.cat(seq,1)
output

tensor([[  5.0000, 160.9702, 157.9619, 292.7193, 235.9922,   0.9994,   0.9109,
           2.0000],
        [  5.0000,  19.1932, 172.2979, 118.8749, 245.4051,   0.9974,   0.9965,
           2.0000],
        [  5.0000, -27.2960, 272.9604, 435.2546, 322.3747,   0.9865,   1.0000,
           2.0000],
        [  5.0000, 364.7225, 136.0251, 377.4218, 160.3831,   0.8234,   0.9997,
           9.0000],
        [  5.0000, 374.9868, 135.9671, 390.3168, 160.1495,   0.5790,   0.9998,
           9.0000]])